# Plotting Multiple Data Series

Complete the following set of exercises to solidify your knowledge of plotting multiple data series with pandas, matplotlib, and seaborn. Part of the challenge that comes with plotting multiple data series is transforming the data into the form needed to visualize it like you want. For some of the exercises in this lab, you will need to transform the data into the form most appropriate for generating the visualization and then create the plot.

In [0]:
%%capture cap_out
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import scipy.constants as k
!pip install ipympl
warnings.filterwarnings('ignore')
%matplotlib widget

In [0]:
#del plt

In [19]:
data = pd.read_csv('https://raw.githubusercontent.com/eduardoheros/lab-plotting-multiple-data-series/master/data/liquor_store_sales.csv')
data.head()

,Year,Month,Supplier,ItemCode,Description,ItemType,RetailSales,RetailTransfers,WarehouseSales
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


## 1. Create a bar chart with bars for total Retail Sales, Retail Transfers, and Warehouse Sales by Item Type.

In [8]:
plt.style.use("fast")
phi=k.golden_ratio
plt.figure(figsize=(phi*7,7))
plt.bar([1,2,3],[data["RetailSales"].sum(),data["RetailTransfers"].sum(),data["WarehouseSales"].sum()],color=["#ff6f69","#ffcc5c","#88d8b0"])
plt.title("Sales")
plt.xticks([1,2,3],["Retail","RetailTransfers","Warehouse"])
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Create a horizontal bar chart showing sales mix for the top 10 suppliers with the most total sales. 

In [9]:
data["Total"]=data.RetailSales+data.RetailTransfers+data.WarehouseSales
filtered=data.groupby("Supplier")["Total"].sum()
filtered.sort_values(ascending=False,inplace=True)
arange=np.arange(10,0,-1)
colors=["#96ceb4","#ffeead","#ff6f69","#ffcc5c","#88d8b0","#96ceb4","#ffeead","#ff6f69"]
plt.style.use("ggplot")
plt.figure(figsize=(7*phi,7))
plt.barh(arange,filtered[0:10],color=colors)
plt.title("Top 10 Sales")
plt.yticks(arange,filtered[0:10].index)
plt.show();


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3. Create a multi-line chart that shows average Retail Sales, Retail Transfers, and Warehouse Sales per month over time.

In [10]:
filtered=data.groupby(["Year","Month"],as_index=False).agg({"RetailSales":"mean","RetailTransfers":"mean","WarehouseSales":"mean"})
filtered

,Year,Month,RetailSales,RetailTransfers,WarehouseSales
0,2017,4,0.000000,15.707503,0.000000
1,2017,5,7.038838,7.421817,27.310548
2,2017,6,7.143914,6.950396,27.839069
3,2017,8,6.409991,6.584726,28.122641
4,2017,9,6.757254,6.419721,22.817909
5,2017,10,6.549021,6.827827,22.289367
6,2017,11,6.765496,7.103699,23.348862
7,2017,12,9.078241,8.353759,21.169463
8,2018,1,5.679413,5.574833,19.072137
9,2018,2,5.939247,6.050136,20.229658


In [0]:
##Se desea realizar manipulación fechas, y resulta más sencillo crear columna con valores tipo str para realizar un ordenamiento
def crea_t(dataframe):
  dataframe["t"]=dataframe.Year.map(str)+"-"+dataframe.Month.map(lambda x:str(0)+str(x) if x<10 else str(x))
  dataframe.drop(["Year","Month"],axis=1,inplace=True)
  return dataframe

In [12]:
crea_t(filtered)
filtered.plot(x="t",y=["RetailSales","RetailTransfers","WarehouseSales"],color=["#ff6f69","#ffcc5c","#88d8b0"],figsize=(7*phi,7))
plt.title("Ventas en 1 año");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
filtered

,RetailSales,RetailTransfers,WarehouseSales,t
0,0.000000,15.707503,0.000000,2017-04
1,7.038838,7.421817,27.310548,2017-05
2,7.143914,6.950396,27.839069,2017-06
3,6.409991,6.584726,28.122641,2017-08
4,6.757254,6.419721,22.817909,2017-09
5,6.549021,6.827827,22.289367,2017-10
6,6.765496,7.103699,23.348862,2017-11
7,9.078241,8.353759,21.169463,2017-12
8,5.679413,5.574833,19.072137,2018-01
9,5.939247,6.050136,20.229658,2018-02


## 4. Plot the same information as above but as a bar chart.

In [14]:
#filtered.set_index("t",inplace=True)
filtered=filtered[["RetailSales","RetailTransfers","WarehouseSales"]]
filtered.plot.bar(color=["#ff6f69","#ffcc5c","#88d8b0"],figsize=(6*phi,6),width=0.85);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 5. Create a multi-line chart that shows Retail Sales summed by Item Type over time (Year & Month).

*Hint: There should be a line representing each Item Type.*

In [29]:
'''
filtered=data.copy()
crea_t(filtered)
filtered=filtered.groupby(["ItemType","t"],as_index=False)["RetailSales"].sum()
tipos=filtered.ItemType.unique()
#for i in range(len(tipos)):
#  f=filtered[filtered["ItemType"]==tipos[i]]
 
f=filtered[filtered["ItemType"]==tipos[0]]
#plt.figure(figsize=(5,5))
plt.plot("t","RetailSales",data=f,color="b")
plt.show()'''
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
 
# Data
df=pd.DataFrame({'x': range(1,11), 'y1': np.random.randn(10), 'y2': np.random.randn(10)+range(1,11), 'y3': np.random.randn(10)+range(11,21) })
 
# multiple line plot
plt.plot( 'x', 'y1', data=df, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)
plt.plot( 'x', 'y2', data=df, marker='', color='olive', linewidth=2)
plt.plot( 'x', 'y3', data=df, marker='', color='olive', linewidth=2, linestyle='dashed', label="toto")
plt.legend()


## 6. Plot the same information as above but as a bar chart.

## 7. Create a scatter plot showing the relationship between Retail Sales (x-axis) and Retail Transfers (y-axis) with the plot points color-coded according to their Item Type.

*Hint: Seaborn's lmplot is the easiest way to generate the scatter plot.*

## 8. Create a scatter matrix using all the numeric fields in the data set with the plot points color-coded by Item Type.

*Hint: Seaborn's pairplot may be your best option here.*